In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import find_peaks
import sys
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/everglow/ubfc-rppg/metadata_hr_csv_path.csv')
df

,id,path
0,1,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
1,3,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
2,4,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
3,5,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
4,8,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
5,9,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
6,10,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
7,11,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
8,12,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...
9,13,/content/drive/MyDrive/everglow/ubfc-rppg/HR-n...


In [ ]:
# frame_counts = []
# for _, row in df.iterrows():
#     with np.load(row['path'], allow_pickle=True) as data:
#         video = data['video']
#         frame_counts.append(video.shape[0])  # 비디오 시퀀스의 길이 (프레임 수) 추가

# 최소 프레임 수 찾기
# min_frames = min(frame_counts)
min_frames = 1368
print(f"Minimum number of frames in the dataset: {min_frames}")

Minimum number of frames in the dataset: 1368


In [ ]:
def load_dataset(df):
    video_data = []
    ppg_data = []
    hr_data = []

    for _, row in df.iterrows():
        with np.load(row['path']) as data:
            video = data['video']
            wave = data['wave']
            hr = data['hr']
            if video.shape[0] > min_frames:
                video = video[:min_frames]
                wave = wave[:min_frames]
                hr = hr[:min_frames]

            video_data.append(video)
            ppg_data.append(wave)
            hr_data.append(hr)
            ## 모델 입력을 HR로 할 결루
            # heart_rate = calculate_hr(wave)
            # ppg_data.append(heart_rate)

    return np.array(video_data), np.array(hr_data)

video_data, ppg_data = load_dataset(df)

In [ ]:
# class ConvBNReLU3D(nn.Module):
#     def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, padding=1):
#         super(ConvBNReLU3D, self).__init__()
#         self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
#         self.bn = nn.BatchNorm3d(out_planes)
#         self.relu = nn.ReLU(inplace=True)

#     def forward(self, x):
#         return self.relu(self.bn(self.conv(x)))

# class MobileNet3D(nn.Module):
#     def __init__(self, input_shape=(1368, 64, 64, 3), num_classes=min_frames):
#         super(MobileNet3D, self).__init__()
#         self.layer1 = ConvBNReLU3D(input_shape[3], 32, stride=2, padding=1)
#         self.layer2 = ConvBNReLU3D(32, 64, padding=1)
#         self.layer3 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)

#         # Add more layers as needed...

#         self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
#         self.fc = nn.Linear(64, num_classes)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)

#         # Forward pass through additional layers...

#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc(x)
#         return x

# # Create the model
# model = MobileNet3D(input_shape=(1368, 64, 64, 3), num_classes=1)
# print(model)

# # For regression, you might use MSE Loss
# criterion = nn.MSELoss()

# # Example input tensor
# input_tensor = torch.randn((1, 3, 1368, 64, 64))  # Batch size of 1
# output = model(input_tensor)
# print(output.shape)  # Expected shape: [1, 1] since num_classes=1


In [ ]:
class HardSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0) / 6.0

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc1 = nn.Conv3d(in_channels, in_channels // reduction, kernel_size=1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv3d(in_channels // reduction, in_channels, kernel_size=1)
        self.hsigmoid = nn.Hardsigmoid()

    def forward(self, x):
        out = self.pool(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.hsigmoid(out)
        return x * out

class ConvBNHardSwish(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, padding=1):
        super(ConvBNHardSwish, self).__init__()
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm3d(out_planes)
        self.activation = HardSwish()

    def forward(self, x):
        return self.activation(self.bn(self.conv(x)))

class MobileNetV3(nn.Module):
    def __init__(self, input_shape=(1368, 64, 64, 3), num_classes=min_frames):
        super(MobileNetV3, self).__init__()
        self.layer1 = ConvBNHardSwish(input_shape[3], 16, stride=2, padding=1)
        self.layer2 = ConvBNHardSwish(16, 16, padding=1)
        self.layer3 = SEBlock(16)
        self.layer4 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        # 이후 레이어를 추가하고 필요에 따라 SEBlock 및 HardSwish 적용...

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(16, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # 추가 레이어를 통한 포워드 패스...

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
# 데이터를 학습, 검증, 테스트 세트로 분할
video_train, video_test, ppg_train, ppg_test = train_test_split(video_data, ppg_data, test_size=0.2, random_state=42)
video_train, video_val, ppg_train, ppg_val = train_test_split(video_train, ppg_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

print(f"video_train shape: {video_train.shape}")
print(f"video_val shape: {video_val.shape}")
print(f"video_test shape: {video_test.shape}")
print(f"ppg_train shape: {ppg_train.shape}")
print(f"ppg_val shape: {ppg_val.shape}")
print(f"ppg_test shape: {ppg_test.shape}")

video_train shape: (24, 1368, 64, 64, 3)
video_val shape: (9, 1368, 64, 64, 3)
video_test shape: (9, 1368, 64, 64, 3)
ppg_train shape: (24, 1368)
ppg_val shape: (9, 1368)
ppg_test shape: (9, 1368)


In [ ]:
class VideoPPGDataset(Dataset):
    def __init__(self, video_data, ppg_data):
        """
        video_data: 비디오 데이터 배열 (샘플 수, 프레임 수, 높이, 너비, 채널)
        ppg_data: PPG 신호 데이터 배열 (샘플 수, 프레임 수)
        """
        self.video_data = video_data
        self.ppg_data = ppg_data

    def __len__(self):
        return len(self.video_data)

    def __getitem__(self, idx):
        video = self.video_data[idx]
        ppg = self.ppg_data[idx]
        return torch.from_numpy(video).permute(3, 0, 1, 2).float(), torch.from_numpy(ppg).float()  # 채널 첫 번째로 이동

# 데이터셋 인스턴스 생성
train_dataset = VideoPPGDataset(video_train, ppg_train)
val_dataset = VideoPPGDataset(video_val, ppg_val)
test_dataset = VideoPPGDataset(video_test, ppg_test)

# DataLoader 인스턴스 생성
batch_size = 8  # 예시로 배치 크기를 4로 설정

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
first_batch_video, first_batch_ppg = next(iter(train_loader))

print("First batch video shape:", first_batch_video.shape)
print("First batch PPG shape:", first_batch_ppg.shape)

First batch video shape: torch.Size([8, 3, 1368, 64, 64])
First batch PPG shape: torch.Size([8, 1368])


In [ ]:
# 위에서 정의한 MobileNet3D 모델 인스턴스화
model = MobileNetV3(input_shape=(1368, 64, 64, 3), num_classes=min_frames)
model.cuda()  # GPU 사용 설정

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.MSELoss()  # 회귀 문제이므로 Mean Squared Error 손실 함수 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 학습률은 실험적으로 조정

# 훈련 루프
num_epochs = 10000  # 에폭 수는 실험적으로 조정

for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for video, ppg in train_loader:
        video = video.cuda()  # 입력 데이터를 GPU로 이동
        ppg = ppg.cuda()  # 타겟 데이터를 GPU로 이동

        optimizer.zero_grad()  # 그래디언트 초기화

        outputs = model(video)  # 모델의 순전파
        loss = criterion(outputs, ppg.unsqueeze(1))  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 최적화

        running_loss += loss.item() * video.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # 검증 루프
    model.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for video, ppg in val_loader:
            video = video.cuda()
            ppg = ppg.cuda()

            outputs = model(video)
            loss = criterion(outputs, ppg.unsqueeze(1))

            val_loss += loss.item() * video.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 1, 1368])) that is different to the input size (torch.Size([8, 1368])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1, 1368])) that is different to the input size (torch.Size([1, 1368])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
Epoch 5001, Train Loss: 595.4276, Val Loss: 228.6072
Epoch 5002, Train Loss: 636.7734, Val Loss: 172.1490
Epoch 5003, Train Loss: 625.2060, Val Loss: 171.1980
Epoch 5004, Train Loss: 595.5276, Val Loss: 176.9480
Epoch 5005, Train Loss: 612.2348, Val Loss: 212.5731
Epoch 5006, Train Loss: 625.9302, Val Loss: 196.9124
Epoch 5007, Train Loss: 579.0916, Val Loss: 187.7225
Epoch 5008, Train Loss: 611.4926, Val Loss: 231.1128
Epoch 5009, Train Loss: 609.1077, Val Loss: 249.2121
Epoch 5010, Train Loss: 562.3197, Val Loss: 195.8646
Epoch 5011, Train Loss: 625.4269, Val Loss: 183.5852
Epoch 5012, Train Loss: 618.3223, Val Loss: 178.2148
Epoch 5013, Train Loss: 606.8198, Val Loss: 178.3981
Epoch 5014, Train Loss: 605.0153, Val Loss: 222.3175
Epoch 5015, Train Loss: 607.5606, Val Loss: 283.3474
Epoch 5016, Train Loss: 606.3025, Val Loss: 242.0696
Epoch 5017, Train Loss: 612.8688, Val Loss: 216.7557
Epoch 5018, Train Loss: 611.1920, Val Loss: 192.